In [1]:
# initialize variables
N = 8 
B = 0.1
number_of_layers = 20

In [2]:
import numpy as np
import torch
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import pandas as pd
from scipy.linalg import expm
import pickle
import time
import scipy
Q = pow(2, N)  # Dimension of the Hilbert space for N spins

def D2B(num, digit):
    string = f'{num:0{digit}b}'  # Convert to binary string with leading zeros
    result = np.array([int(ele) for ele in string], int)  # Convert each bit to an integer array
    return result

def B2D(array):
    res = 0
    for ele in array:
        res = (res << 1) | ele  # Left shift and add the current bit
    return res

def spin_reflection(array):
    return 1 - array  # Flip each spin (1 -> 0, 0 -> 1)

# This one is the one with critial energy gap 0.001
J_matrix= np.array([[0.0, -0.770026, -0.612588, 1.884469, -0.570562, 0.616424, 1.663316, -0.448101],
                                         [0.0, 0.0, 0.795786, 0.693566, -1.818596, -1.793477, 0.869296, -0.476921],
                                         [0.0, 0.0, 0.0, 1.166175, 2.639218, 0.896904, -0.685189, 0.523481],
                                         [0.0, 0.0, 0.0, 0.0, 1.069138, 0.163168, 0.637887, 0.75839],
                                         [0.0, 0.0, 0.0, 0.0, 0.0, 1.210793, 0.186866, 0.464955],
                                         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -2.255085, -1.129867],
                                         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.741586],
                                         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])


# This one is the one with critial energy gap 0.009
#J_matrix = np.array([[0.000000, 0.773813, 2.485213, -0.561542, 0.008873, 0.691680, 0.086291, -0.564034], [0.000000, 0.000000, 1.397455, -0.112515, 0.388229, -1.835349, 1.372802, 0.738212], [0.000000, 0.000000, 0.000000, -0.169025, 0.746900, -1.199308, 0.269533, -1.233924], [0.000000, 0.000000, 0.000000, 0.000000, 0.939910, 0.653794, 1.202234, 0.980422], [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, -1.008154, 2.553466, 0.876100], [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.196304, -0.398031], [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.282909], [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000]])
# Hamiltonian construction
H_a = np.zeros(Q)  # Initialize the interaction term (ZZ term), which is diagonal
HB_matrix = np.zeros((Q, Q))  # Initialize the transverse field term (X term)

for state in range(Q):  # Loop over all possible basis states
    basis = D2B(state, N)  # Convert the current state index to a binary representation
    for i in range(N):  # Loop over each spin in the state
        flipped_basis = basis.copy()  # Copy the current basis state
        flipped_basis[i] = 1 - flipped_basis[i]  # Flip the i-th spin
        new_state = B2D(flipped_basis)  # Convert the flipped basis back to a decimal index
        HB_matrix[state, new_state] += 1  # Add the contribution to the X term

    # Interaction term (ZZ term)
    for i in range(N):  # Loop over each spin
        for j in range(i + 1, N):  # Loop over pairs of spins (i, j)
            J_ij = J_matrix[i][j]  # Interaction strength between spin i and j
            # Add the contribution to the ZZ term
            H_a[state] += J_ij * (1 if basis[i] == basis[j] else -1)

# Convert H_a to a diagonal matrix
HA_matrix = np.diag(H_a)

In [3]:
def calculate_spin_parity(eigenvector):
    parity_sum = 0  # Initialize the parity sum for the current eigenvector
    for basis_idx in range(Q):  # Loop over all basis states
        basis = D2B(basis_idx, N)  # Convert basis index to binary representation
        reflected_basis = spin_reflection(basis)  # Apply spin reflection to the basis state
        reflected_idx = B2D(reflected_basis)  # Convert the reflected basis back to a decimal index

        # Calculate the parity by comparing original and reflected amplitudes
        parity_sum += eigenvector[basis_idx] * eigenvector[reflected_idx]

    return round(parity_sum, 5)  # Round to 5 digits

In [4]:
# get true ground state 
w, v = eigh(HA_matrix + B* HB_matrix)
inx = np.argsort(w)
true_gs_vec = v[:, inx[0]]
true_gs_energy = w[0]
E_max = w[-1]

In [5]:
print('true ground state energy is', true_gs_energy)

true ground state energy is -13.609517686814241


In [6]:
# get  ground state with initial H_B Hamiltonian 
w, v = eigh(HB_matrix)
inx = np.argsort(w)
temp_vec = v[:, inx[0]] # the index here could be zero or one, depending on the parity
# but if the wrong index will always give roughly zero fidelity 

In [7]:
calculate_spin_parity(temp_vec)

1.0

In [8]:
# get initial fidelity
fidelity_t_0 = pow(np.dot(true_gs_vec,temp_vec),2) # all real so no need to c.c.
print('fidelity before optim is', fidelity_t_0)

fidelity before optim is 0.020315997322665695


In [9]:
# move to gpu 

HA_matrix = torch.tensor(HA_matrix, dtype=torch.complex128).to(device=device)
HB_matrix = torch.tensor(HB_matrix, dtype=torch.complex128).to(device=device)
target_H = HA_matrix + B * HB_matrix
true_gs_vec = torch.tensor(true_gs_vec, dtype=torch.complex128, device=device)
temp_vec = torch.tensor(temp_vec, dtype=torch.complex128, device=device)

In [10]:
def get_fidelity_torch(vector1, vector2):
    overlap = torch.tensordot(vector1, vector2, dims=1)
    overlap_conj = torch.conj(overlap)
    fidelity = torch.real(overlap * overlap_conj)
    return fidelity

def get_energy_torch(matrix, vector):
    conj_vector = torch.conj(vector)
    energy = torch.real(torch.tensordot(conj_vector, torch.tensordot(matrix, vector, dims=1), dims=1)).to(torch.float64)
    return energy

In [11]:
def lambda_evolve(x):
    # product state
    p = temp_vec  # p stands for product
    
    # loop each time step
    for i in range(number_of_layers):
        H_matrix = HA_matrix + x[i + number_of_layers] * HB_matrix
        exp_imgH_matrix = torch.matrix_exp(-1j * x[i] * H_matrix)
        new_p = torch.matmul(exp_imgH_matrix, p)
        # update state
        p = new_p
    
    # get energy
    energy = get_energy_torch(target_H, p)
    return energy

In [12]:
def lambda_evolve_fidelity_list(x):
    fidelity_list = []
    inst_f_list = []
    p = temp_vec  # p stands for product
     
    # loop each time step
    for i in range(number_of_layers):            
        H_matrix = HA_matrix + x[i + number_of_layers] * HB_matrix
        # inst H
        if x[i] < 0 and x[i + number_of_layers] < 0:
            print('both smaller than zero', i)
            w, v = torch.linalg.eigh(-HA_matrix - x[i + number_of_layers] * HB_matrix)
            inst_gs_vec = v[:, 0]
        else:
            w, v = torch.linalg.eigh(HA_matrix + x[i + number_of_layers] * HB_matrix)
            inst_gs_vec = v[:, 0]
            
        exp_imgH_matrix = torch.matrix_exp(-1j * x[i] * H_matrix)
        new_p = torch.matmul(exp_imgH_matrix, p)
        # update state
        p = new_p
        fidelity_list.append(get_fidelity_torch(true_gs_vec, p))
        inst_f_list.append(get_fidelity_torch(inst_gs_vec, p))
    
    return torch.tensor(fidelity_list), torch.tensor(inst_f_list)

In [13]:
def exp_decay_guess_new(B_0, B_f, j):
    tau = np.log(B_0 / B_f)
    value = B_0 * np.exp(-j * tau / (number_of_layers - 1))  # Adjust for exact B_0 and B_f
    return value

# Parameters
B_0 = 10                 # Initial value (starts here)
B_f = 0.1               # Final value (ends here)
B_guess = np.array([exp_decay_guess_new(B_0, B_f, layer) for layer in range(number_of_layers)])
lambada_guess = np.random.uniform(1,2,number_of_layers)
x = np.concatenate((lambada_guess, B_guess))
x =torch.tensor(x, dtype=torch.float64, device=device, requires_grad=True)
######################################################################################################
# the first len(number_of_layers) parameters are the lambda field parameter while the rest is for the B field 

In [14]:
# Optimizer setup
def run_optimization(x, lr, max_epochs, gradient_threshold):
    optimizer = optim.LBFGS([x], lr=lr, max_iter=20, history_size=1000)
    epoch = 0

    def closure():
        optimizer.zero_grad()
        energy = lambda_evolve(x)
        energy.backward()
        return energy

    start_time = time.time()
    while epoch < max_epochs:
        optimizer.step(closure)

        # Log and save data
        gradient_norm = x.grad.norm().cpu().detach().numpy()
        print(f"Epoch {epoch + 1}, Energy: {lambda_evolve(x).cpu().detach().numpy():.6f}, Gradient Norm: {gradient_norm:.6f}")

        # Check if gradient is smaller than threshold
        if gradient_norm < gradient_threshold:
            break

        epoch += 1
    
    end_time = time.time()
    print(f"Total time: {end_time - start_time:.2f} seconds")

In [ ]:
# Run the optimization
run_optimization(x, 0.1, 10000, 0.001)

Epoch 1, Energy: -13.435045, Gradient Norm: 0.795009
Epoch 2, Energy: -13.490161, Gradient Norm: 0.135316
Epoch 3, Energy: -13.511200, Gradient Norm: 0.130702
Epoch 4, Energy: -13.525549, Gradient Norm: 0.133868
Epoch 5, Energy: -13.528404, Gradient Norm: 0.024852
Epoch 6, Energy: -13.531505, Gradient Norm: 0.041655
Epoch 7, Energy: -13.533866, Gradient Norm: 0.022868
Epoch 8, Energy: -13.534335, Gradient Norm: 0.015694
Epoch 9, Energy: -13.534426, Gradient Norm: 0.003975
Epoch 10, Energy: -13.534475, Gradient Norm: 0.002577
Epoch 11, Energy: -13.534484, Gradient Norm: 0.001128


In [ ]:
torch.set_printoptions(precision=14)
lambda_evolve(x)